In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# The GPU id to use, usually either "0" or "1";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

# TensorFlow and tf.keras
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.10.0


In [5]:
batch_size = 128
num_classes = 10
epochs = 12

# the data, split between train and test sets
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
# input image dimensions
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [6]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [7]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [8]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [10]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 7s 116us/step - loss: 0.7395 - acc: 0.7334 - val_loss: 0.5087 - val_acc: 0.8103
Epoch 2/12
60000/60000 [==============================] - 5s 80us/step - loss: 0.4092 - acc: 0.8523 - val_loss: 0.4433 - val_acc: 0.8360
Epoch 3/12
60000/60000 [==============================] - 5s 79us/step - loss: 0.3244 - acc: 0.8821 - val_loss: 0.3365 - val_acc: 0.8801
Epoch 4/12
60000/60000 [==============================] - 5s 79us/step - loss: 0.2785 - acc: 0.8968 - val_loss: 0.3187 - val_acc: 0.8808
Epoch 5/12
60000/60000 [==============================] - 5s 79us/step - loss: 0.2444 - acc: 0.9110 - val_loss: 0.2807 - val_acc: 0.8954
Epoch 6/12
60000/60000 [==============================] - 5s 79us/step - loss: 0.2150 - acc: 0.9211 - val_loss: 0.2831 - val_acc: 0.8962
Epoch 7/12
60000/60000 [==============================] - 5s 80us/step - loss: 0.1888 - acc: 0.9312 - val_loss: 0.2798 - val_ac

In [11]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

10000/10000 [==============================] - 1s 71us/step
Test accuracy: 0.9137


## Make predictions

In [12]:
predictions = model.predict(x_test)

In [13]:
predictions[0]

array([2.1552807e-05, 5.1608631e-06, 8.3366922e-06, 8.3626713e-07,
       2.3849253e-07, 2.8720462e-05, 5.5378507e-07, 1.5145341e-04,
       4.0970977e-07, 9.9978286e-01], dtype=float32)

In [14]:
np.argmax(predictions[0])

9

In [15]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.], dtype=float32)

In [16]:
# serialize model to JSON
model_json = model.to_json()
with open("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CNN.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/home/jiameng/packages/ReachNN/ReachNN-CNN/model/model_CNN.h5")
print("Saved model to disk")

Saved model to disk


In [38]:
weights = []
for layer in model.layers:
    weights.append(layer.get_weights()) # list of numpy arrays